<a href="https://colab.research.google.com/github/mille-s/Text_Structuring/blob/main/Thiago_Pipeline_Eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare folders and files

In [ ]:
#@title Prepare repo (SM)
# IPython.display allows clearing the output in a Jupyter notebook cell
from IPython.display import clear_output
import os

# Clone original repo
! git clone https://github.com/ThiagoCF05/DeepNLG.git

# Clone repo for conversion of parser output to format needed for evaluation
! git clone 'https://github.com/mille-s/Text_Structuring.git'
mflens_data_folder = '/content/Text_Structuring/data'
if not os.path.exists(mflens_data_folder):
  os.makedirs(mflens_data_folder)

# Make new output folder
# os.makedirs('out_SM')

# Requirements
# Some get installed, some don't
# ! pip install absl-py==0.7.1
# ! pip install astor==0.8.0
# ! pip install certifi==2019.3.9
# ! pip install chardet==3.0.4
# ! pip install Cython==0.29.7
# ! pip install dyNET==2.1
# ! pip install gast==0.2.2
# ! pip install grpcio==1.20.1
# ! pip install h5py==2.9.0
# ! pip install idna==2.8
# ! pip install Keras-Applications==1.0.7
# ! pip install Keras-Preprocessing==1.0.9
# ! pip install Markdown==3.1.1
# ! pip install mock==3.0.5
# ! pip install numpy==1.16.3
# ! pip install protobuf==3.7.1
# ! pip install psutil==5.6.6
# ! pip install requests==2.22.0
# ! pip install six==1.12.0
# ! pip install stanfordcorenlp==3.9.1.1
# ! pip install tensorboard==1.13.1
# ! pip install tensorflow==2.5.1
# ! pip install tensorflow-estimator==1.13.0
# ! pip install termcolor==1.1.0
# ! pip install urllib3==1.26.5
# ! pip install Werkzeug==0.15.4
clear_output()
print('Repo ready!')


In [ ]:
#@title Process scripts (SM)
# Adapt import paths to Colab repo
import codecs

# Define paths
path_process_ordering = '/content/DeepNLG/ordering/preprocess.py'
path_process_structuring = '/content/DeepNLG/structing/preprocess.py'
path_data = '/content/DeepNLG/versions/v1.4/en'
path_out_ordering = 'out_SM/ordering'
path_out_structuring = 'out_SM/structing'

preproc_order_code_lines = codecs.open(path_process_ordering, 'r', 'utf-8').readlines()
preproc_structing_code_lines = codecs.open(path_process_structuring, 'r', 'utf-8').readlines()

path_process_ordering_Colab = '/content/DeepNLG/ordering/preprocess_Colab.py'
path_process_structuring_Colab = '/content/DeepNLG/structing/preprocess_Colab.py'

with codecs.open(path_process_ordering_Colab, 'w', 'utf-8') as fo_order:
  for line in preproc_order_code_lines:
    if line == 'import load\n':
      fo_order.write('from content.DeepNLG import load\n')
    elif line == 'import parsing\n':
      fo_order.write('from content.DeepNLG import parsing\n')
    elif line == 'import utils\n':
      fo_order.write('from content.DeepNLG import utils\n')
    elif line == 'from superpreprocess import Preprocess\n':
      fo_order.write('from content.DeepNLG.superpreprocess import Preprocess\n')
    else:
      fo_order.write(line)

with codecs.open(path_process_structuring_Colab, 'w', 'utf-8') as fo_struct:
  for line in preproc_order_code_lines:
    if line == 'import load\n':
      fo_struct.write('from content.DeepNLG import load\n')
    elif line == 'import parsing\n':
      fo_struct.write('from content.DeepNLG import parsing\n')
    elif line == 'import utils\n':
      fo_struct.write('from content.DeepNLG import utils\n')
    elif line == 'from superpreprocess import Preprocess\n':
      fo_struct.write('from content.DeepNLG.superpreprocess import Preprocess\n')
    else:
      fo_struct.write(line)

In [ ]:
#@title Make reference file for joint eval of ordering and structuring (SM)

path_ordANDstructRefCode = '/content/Text_Structuring/code/orderANDstruct_makeRefFile.py'
path_gold_structuring = '/content/DeepNLG/evaluation/data/structing/test.json'

! python {path_ordANDstructRefCode} {path_gold_structuring}

In [ ]:
#@title Extract Thiago's predicted files for joint eval of ordering and structuring (SM)
import os

path_ordANDstructPredCode = '/content/Text_Structuring/code/orderANDstruct_makePredFiles.py'
path_gold_structuring = '/content/DeepNLG/evaluation/data/structing/test.json'
path_pred = '/content/DeepNLG/evaluation/results'
# path_pred = '/content/Text_Structuring/data'

# model_name = 'transformer'#@param['major', 'random', 'rnn', 'transformer']
model_names = ['major', 'random', 'rnn', 'transformer']

for model_name in model_names:
  ! python {path_ordANDstructPredCode} {path_gold_structuring} {path_pred} {model_name}

In [ ]:
#@title Extract ordering and structuring info from FORGe intermediate reps (drag and drop file in /content/Text_Structuring/data) (SM)

path_forge2castro_code = '/content/Text_Structuring/code/forge2castro.py'
forge_out_file = 'none' #@param['11-SMorphText.conllu', 'none']

if not forge_out_file == 'none':
  ! python {path_forge2castro_code} {mflens_data_folder} {forge_out_file}


In [ ]:
#@title Add new outputs (SM, drag and drop files [task]_[system_name]-test.out.postprocessed in /content/Text_Structuring/data)
import os
import glob
import sys

# I need 3 files for the evaluation (example with 2AO files)
# - ordering_2AO-test.out.postprocessed (for evaluation of ordering only)
# - structuring_2AO-lin-test.out.postprocessed (for evaluation of structuring only)
# - structuring_2AO-test.out.postprocessed ( for evaluation of ordering AND structuring together)

files_in = mflens_data_folder

def create_folder(folder_path):
  if not os.path.exists(folder_path):
    os.makedirs(folder_path)

files_in_paths = glob.glob(os.path.join(files_in, '*.postprocessed'))
for file_in_path in files_in_paths:
  head, tail = os.path.split(file_in_path)
  task = tail.split('_', 1)[0]
  if not (task == 'ordering' or task == 'structuring'):
    sys.exit(f'Task in input file name not recognised!')
  new_file_name = tail.rsplit('-', 1)[1]
  if not new_file_name == 'test.out.postprocessed':
    sys.exit(f'The file name should end with -test.out.postprocessed!')
  system_name = tail.split('_', 1)[1].rsplit('-', 1)[0]
  system = ''
  if system_name.startswith('2') or system_name.startswith('3'):
    system = 'mflens'
  else:
    system = system_name
  orderingFolder = os.path.join(('/content/DeepNLG/evaluation/results/steps/ordering'), system)
  create_folder(orderingFolder)
  srtructuringFolder = os.path.join(('/content/DeepNLG/evaluation/results/steps/structing'), system)
  create_folder(srtructuringFolder)
  orderingANDsrtructuringFolder = os.path.join(('/content/DeepNLG/evaluation/results/steps/orderingANDstructing'), system)
  create_folder(orderingANDsrtructuringFolder)

  # to copy a file to a different location with the same name cp file new/location/file
  if task == 'ordering':
    dest_path = os.path.join(orderingFolder, new_file_name)
    ! cp {file_in_path} {dest_path}
    print(f'Copied file {dest_path} from {file_in_path}')
  elif task == 'structuring':
    if tail.split('-')[1] == 'lin':
      dest_path = os.path.join(srtructuringFolder, new_file_name)
      ! cp {file_in_path} {dest_path}
      print(f'Copied file {dest_path} from {file_in_path}')
    else:
      dest_path = os.path.join(orderingANDsrtructuringFolder, new_file_name)
      ! cp {file_in_path} {dest_path}
      print(f'Copied file {dest_path} from {file_in_path}')

In [ ]:
#@title Extract reference data (SM, not used)
# Running this code, I don't obtain exactly the same data as in the repo; at least the order of the datapoints is different (haven't checked the contents; filesize is the same).
# ! python {path_process_ordering_Colab} {path_data} {path_out_ordering}
# ! python {path_process_structuring_Colab} {path_data} {path_out_structuring}

## Evaluation of Ordering and Structuring

In [ ]:
#@title Set folder path and unseen domains (TCF)
import json
import os
import nltk
import numpy as np
import subprocess

unseen_domains = ['Artist', 'Politician', 'CelestialBody', 'Athlete', 'MeanOfTransportation']
# root path
path='/content/DeepNLG/evaluation/'

In [ ]:
#@title Choose datasets, tasks and models to evaluate (SM, add new models here)

def addToList(list_to_fill, param, value):
  if param == True:
    list_to_fill.append(value)

# dev is not supported for some tasks
data_list = ['test']#['dev', 'test']
# data_dev = False#@param{type:"boolean"}
# addToList(data_list, data_dev, 'dev')
# data_test = True#@param{type:"boolean"}
# addToList(data_list, data_test, 'test')

task_list = []#['ordering', 'structing', 'orderingANDstructing']
task_ordering = True#@param{type:"boolean"}
addToList(task_list, task_ordering, 'ordering')
task_structuring = True#@param{type:"boolean"}
addToList(task_list, task_structuring, 'structing')
task_orderingANDstructuring = False#@param{type:"boolean"}
addToList(task_list, task_orderingANDstructuring, 'orderingANDstructing')

model_list = []#['major', 'rnn', 'transformer', 'random', 'mflens', 'forge']
model_major = True#@param{type:"boolean"}
addToList(model_list, model_major, 'major')
model_rnn = True#@param{type:"boolean"}
addToList(model_list, model_rnn, 'rnn')
model_transformer = True#@param{type:"boolean"}
addToList(model_list, model_transformer, 'transformer')
model_random = True#@param{type:"boolean"}
addToList(model_list, model_random, 'random')
model_mflens = False#@param{type:"boolean"}
addToList(model_list, model_mflens, 'mflens')
model_forge = False#@param{type:"boolean"}
addToList(model_list, model_forge, 'forge')

In [ ]:
#@title All domains (TCF)

print('All domains:\n-----------')
# model_thiag = 'mflens'#@param['major', 'random', 'transformer', 'rnn', 'mflens' ]
print_inOut_pairs = False#@param {type:"boolean"}

# SM: replaced dataet, task and model lists by variables that we fill in teh cell above
for _set in data_list:

    for task in task_list:
        gold_path=os.path.join(path, 'data', task, _set + '.json')
        gold = json.load(open(gold_path))
        print('\n')

        for model in model_list:
            # SM: added condition (FORGe cannot easily do only structuring, so skipped for now)
            if task == 'structing' and model == 'forge':
              pass
            else:
              p = os.path.join(path, 'results/steps', task, model, _set + '.out.postprocessed')
              with open(p) as f:
                  y_pred_ = f.read().split('\n')[:-1]
              # y_pred: runwayLength 3rd_runway_SurfaceType
              # y_real: ['3rd_runway_SurfaceType runwayLength', 'runwayLength 3rd_runway_SurfaceType']
              y_real, y_pred = [], []
              for i, g in enumerate(gold):
                  t = [' '.join(target['output']) for target in g['targets']]
                  y_real.append(t)
                  y_pred.append(y_pred_[i].strip())

              num, dem = 0.0, 0
              for i, y_ in enumerate(y_pred):
                  y = y_real[i]
                  # count 1 each time a prediction for input X is found in the list of references for X
                  if y_.strip() in y:
                      num += 1
                      # SM: added print
                      if print_inOut_pairs == True:
                          print(f'#{i}[+]: {y_.strip()} - {y}')
                  # SM: added else
                  else:
                      if print_inOut_pairs == True:
                          print(f'#{i}[-]: {y_.strip()} - {y}')
                  # dem = total count of predictions made
                  dem += 1
              print('Task: ', task)
              print('Set: ', _set)
              print('Model: ', model)
              # for c, (pred, ref) in enumerate(zip(y_pred, y_real)):
              #   print(f'#{c}: {y_pred[c]} - {y_real[c]}')
              # print(f'References ({len(y_real)}):')
              # print(y_real)
              # print(f'Predictions ({len(y_pred)}):')
              # print(y_pred)
              print('Accuracy: ', round(num/dem, 2))
              print(10 * '-')

In [ ]:
#@title Seen domains (TCF)
print('Seen domains:\n-----------')

# SM: replaced dataet, task and model lists by variables that we fill in teh cell above
for _set in data_list:

    for task in task_list:
        gold_path=os.path.join(path, 'data', task, _set + '.json')
        gold = json.load(open(gold_path))
        print('\n')

        for model in model_list:
            p = os.path.join(path, 'results/steps', task, model, _set + '.out.postprocessed')
            with open(p) as f:
                y_pred_ = f.read().split('\n')[:-1]

            y_real, y_pred = [], []
            for i, g in enumerate(gold):
                if g['category'] not in unseen_domains:
                    t = [' '.join(target['output']) for target in g['targets']]
                    y_real.append(t)
                    y_pred.append(y_pred_[i].strip())

            num, dem = 0.0, 0
            for i, y_ in enumerate(y_pred):
                y = y_real[i]
                if y_.strip() in y:
                    num += 1
                dem += 1
            print('Task: ', task)
            print('Set: ', _set)
            print('Model: ', model)
            print('Accuracy: ', round(num/dem, 2))
            print(10 * '-')


In [ ]:
#@title Unseen domains (TCF)
print('Unseen domains:\n-----------')

# SM: replaced dataet, task and model lists by variables that we fill in teh cell above
for _set in data_list:

    for task in task_list:
        gold_path=os.path.join(path, 'data', task, _set + '.json')
        gold = json.load(open(gold_path))
        print('\n')

        for model in model_list:
            p = os.path.join(path, 'results/steps', task, model, _set + '.out.postprocessed')
            with open(p) as f:
                y_pred_ = f.read().split('\n')[:-1]

            y_real, y_pred = [], []
            for i, g in enumerate(gold):
                if g['category'] in unseen_domains:
                    t = [' '.join(target['output']) for target in g['targets']]
                    y_real.append(t)

                    y_pred.append(y_pred_[i].strip())

            num, dem = 0.0, 0
            for i, y_ in enumerate(y_pred):
                y = y_real[i]
                if y_.strip() in y:
                    num += 1
                dem += 1
            print('Task: ', task)
            print('Set: ', _set)
            print('Model: ', model)
            print('Accuracy: ', round(num/dem, 2) if dem > 0 else 0)
            print(10 * '-')

## Evaluation of the rest of the pipeline (not tested on Colab)

### Evaluation of Referring Expressions

In [ ]:
print('All domains:')
for _set in ['dev', 'test']:
    gold_path=os.path.join(path, 'data', 'reg', _set + '.json')
    gold = json.load(open(gold_path))

    p = os.path.join(path, 'results/steps/reg', _set + '.out.postprocessed')
    with open(p) as f:
        y_pred_ = f.read().split('\n')[:-1]

    y_real, y_pred, y_baseline = [], [], []
    for i, g in enumerate(gold):
        y_real.append(' '.join(g['refex']).strip().lower())
        y_pred.append(y_pred_[i].strip().lower())

    num, dem = 0.0, 0
    baseline = 0
    for i, y_ in enumerate(y_pred):
        refex = ' '.join(nltk.word_tokenize(gold[i]['entity'].replace('\'', ' ').replace('\"', ' ').replace('_', ' ')))
        y = y_real[i]
        if y_.strip() == y:
            num += 1
        if refex.strip().lower() == y:
            baseline += 1
        dem += 1
    print('REG Task:')
    print('Set: ', _set)
    print('Baseline Accuracy: ', round(baseline/dem, 2) if dem > 0 else 0)
    print('NeuralREG Accuracy: ', round(num/dem, 2) if dem > 0 else 0)
    print(10 * '-')

In [ ]:
print('Seen domains:')
for _set in ['dev', 'test']:
    gold_path=os.path.join(path, 'data', 'reg', _set + '.json')
    gold = json.load(open(gold_path))

    p = os.path.join(path, 'results/steps/reg', _set + '.out.postprocessed')
    with open(p) as f:
        y_pred_ = f.read().split('\n')[:-1]

    y_real, y_pred, y_baseline = [], [], []
    for i, g in enumerate(gold):
    #     if g['category'] in unseen_domains:
        y_real.append(' '.join(g['refex']).strip().lower())
        y_pred.append(y_pred_[i].strip().lower())

    num, dem = 0.0, 0
    baseline = 0
    for i, y_ in enumerate(y_pred):
        if gold[i]['category'] not in unseen_domains:
            refex = ' '.join(nltk.word_tokenize(gold[i]['entity'].replace('\'', ' ').replace('\"', ' ').replace('_', ' ')))
            y = y_real[i]
            if y_.strip() == y:
                num += 1
            if refex.strip().lower() == y:
                baseline += 1
            dem += 1
    print('REG Task:')
    print('Set: ', _set)
    print('Baseline Accuracy: ', round(baseline/dem, 2) if dem > 0 else 0)
    print('NeuralREG Accuracy: ', round(num/dem, 2) if dem > 0 else 0)
    print(10 * '-')

In [ ]:
print('Unseen domains:')
for _set in ['dev', 'test']:
    gold_path=os.path.join(path, 'data', 'reg', _set + '.json')
    gold = json.load(open(gold_path))

    p = os.path.join(path, 'results/steps/reg', _set + '.out.postprocessed')
    with open(p) as f:
        y_pred_ = f.read().split('\n')[:-1]

    y_real, y_pred, y_baseline = [], [], []
    for i, g in enumerate(gold):
        y_real.append(' '.join(g['refex']).strip().lower())
        y_pred.append(y_pred_[i].strip().lower())

    num, dem = 0.0, 0
    baseline = 0
    for i, y_ in enumerate(y_pred):
        if gold[i]['category'] in unseen_domains:
            refex = ' '.join(nltk.word_tokenize(gold[i]['entity'].replace('\'', ' ').replace('\"', ' ').replace('_', ' ')))
            y = y_real[i]
            if y_.strip() == y:
                num += 1
            if refex.strip().lower() == y:
                baseline += 1
            dem += 1
    print('REG Task:')
    print('Set: ', _set)
    print('Baseline Accuracy: ', round(baseline/dem, 2) if dem > 0 else 0)
    print('NeuralREG Accuracy: ', round(num/dem, 2) if dem > 0 else 0)
    print(10 * '-')

### Evaluation of Lexicalization

In [ ]:
print('All domains:')
for _set in ['dev', 'test']:
    for model in ['random', 'major', 'transformer', 'rnn']:
        gold_path=os.path.join(path, 'data', 'lexicalization', _set + '.json')
        gold = json.load(open(gold_path))

        p = os.path.join(path, 'results/steps/lexicalization', model, _set + '.out.postprocessed')
        with open(p) as f:
            y_pred_ = f.read().split('\n')[:-1]

        y_real, y_pred = [], []
        for i, g in enumerate(gold):
        #     if g['category'] in unseen_domains:
            t = [' '.join(target['output']).lower() for target in g['targets']]
            y_real.append(t)
            y_pred.append(y_pred_[i].strip().lower())

        with open('predictions', 'w') as f:
            f.write('\n'.join(y_pred))

        nfiles = max([len(refs) for refs in y_real])
        for i in range(nfiles):
            with open('reference' + str(i+1), 'w') as f:
                for refs in y_real:
                    if i < len(refs):
                        f.write(refs[i])
                    f.write('\n')

        nematus = '/roaming/tcastrof/workspace/nematus/data/multi-bleu.perl'
        command = 'perl ' + nematus + ' reference1 reference2 reference3 reference4 reference5 reference6 reference7 reference8 < predictions'
        result = subprocess.check_output(command, shell=True)
        print('Lexicalization: ')
        print('Set: ', _set)
        print('Model: ', model)
        print(result)
        print(10 * '-')

        os.remove('reference1')
        os.remove('reference2')
        os.remove('reference3')
        os.remove('reference4')
        os.remove('reference5')
        os.remove('reference6')
        os.remove('reference7')
        os.remove('reference8')
        os.remove('predictions')


In [ ]:
print('Seen domains:')
for _set in ['dev', 'test']:
    for model in ['random', 'major', 'transformer', 'rnn']:
        gold_path=os.path.join(path, 'data', 'lexicalization', _set + '.json')
        gold = json.load(open(gold_path))

        p = os.path.join(path, 'results/steps/lexicalization', model, _set + '.out.postprocessed')
        with open(p) as f:
            y_pred_ = f.read().split('\n')[:-1]

        y_real, y_pred = [], []
        for i, g in enumerate(gold):
            if g['category'] not in unseen_domains:
                t = [' '.join(target['output']).lower() for target in g['targets']]
                y_real.append(t)
                y_pred.append(y_pred_[i].strip().lower())

        with open('predictions', 'w') as f:
            f.write('\n'.join(y_pred))

        nfiles = max([len(refs) for refs in y_real])
        for i in range(nfiles):
            with open('reference' + str(i+1), 'w') as f:
                for refs in y_real:
                    if i < len(refs):
                        f.write(refs[i])
                    f.write('\n')

        nematus = '/roaming/tcastrof/workspace/nematus/data/multi-bleu.perl'
        command = 'perl ' + nematus + ' reference1 reference2 reference3 reference4 reference5 reference6 reference7 reference8 < predictions'
        result = subprocess.check_output(command, shell=True)
        print('Lexicalization: ')
        print('Set: ', _set)
        print('Model: ', model)
        print(result)
        print(10 * '-')

        os.remove('predictions')
        os.remove('reference1')
        os.remove('reference2')
        os.remove('reference3')
        os.remove('reference4')
        os.remove('reference5')
        os.remove('reference6')
        os.remove('reference7')
        os.remove('reference8')

In [ ]:
print('Unseen domains:')
for _set in ['dev', 'test']:
    for model in ['random', 'major', 'transformer', 'rnn']:
        gold_path=os.path.join(path, 'data', 'lexicalization', _set + '.json')
        gold = json.load(open(gold_path))

        p = os.path.join(path, 'results/steps/lexicalization', model, _set + '.out.postprocessed')
        with open(p) as f:
            y_pred_ = f.read().split('\n')[:-1]

        y_real, y_pred = [], []
        for i, g in enumerate(gold):
            if g['category'] in unseen_domains:
                t = [' '.join(target['output']).lower() for target in g['targets']]
                y_real.append(t)
                y_pred.append(y_pred_[i].strip().lower())

        with open('predictions', 'w') as f:
            f.write('\n'.join(y_pred))

        try:
            nfiles = max([len(refs) for refs in y_real])
            for i in range(nfiles):
                with open('reference' + str(i+1), 'w') as f:
                    for refs in y_real:
                        if i < len(refs):
                            f.write(refs[i])
                        f.write('\n')

            nematus = '/roaming/tcastrof/workspace/nematus/data/multi-bleu.perl'
            command = 'perl ' + nematus + ' reference1 reference2 reference3 reference4 reference5 reference6 reference7 reference8 < predictions'
            result = subprocess.check_output(command, shell=True)
            print('Lexicalization: ')
            print('Set: ', _set)
            print('Model: ', model)
            print(result)
            print(10 * '-')

            os.remove('predictions')
            os.remove('reference1')
            os.remove('reference2')
            os.remove('reference3')
            os.remove('reference4')
            os.remove('reference5')
            os.remove('reference6')
            os.remove('reference7')
            os.remove('reference8')
        except:
            pass


### Evaluation of Final Texts (BLEU)

In [ ]:
print('All domains:')
for _set in ['dev', 'test']:
    gold_path=os.path.join(path, 'data', 'end2end', _set + '.json')
    gold = json.load(open(gold_path))
    for kind in ['pipeline', 'end2end']:
        for model in ['rand', 'major', 'transformer', 'rnn']:
            if kind == 'end2end' and model in ['rand', 'major']:
                continue
            else:
                p = os.path.join(path, 'results', kind, model, _set + '.out.postprocessed')
                with open(p) as f:
                    y_pred_ = f.read().split('\n')[:-1]

                y_real, y_pred = [], []
                for i, g in enumerate(gold):
                #     if g['category'] in unseen_domains:
                    targets = [nltk.word_tokenize(' '.join(target['output'])) for target in g['targets']]
                    t = [' '.join(target).lower() for target in targets]
                    y_real.append(t)
                    pred = ' '.join(nltk.word_tokenize(y_pred_[i])).lower()
                    y_pred.append(pred)

                with open('predictions', 'w') as f:
                    f.write('\n'.join(y_pred))

                nfiles = max([len(refs) for refs in y_real])
                for i in range(nfiles):
                    with open('reference' + str(i+1), 'w') as f:
                        for refs in y_real:
                            if i < len(refs):
                                f.write(refs[i])
                            f.write('\n')

                nematus = '/roaming/tcastrof/workspace/nematus/data/multi-bleu.perl'
                command = 'perl ' + nematus + ' reference1 reference2 reference3 reference4 reference5 reference6 reference7 reference8 < predictions'
                result = subprocess.check_output(command, shell=True)
                print('Final: ')
                print('Set: ', _set)
                print('Approach:', kind)
                print('Model: ', model)
                print(result)
                print(10 * '-')

                os.remove('reference1')
                os.remove('reference2')
                os.remove('reference3')
                os.remove('reference4')
                os.remove('reference5')
                os.remove('reference6')
                os.remove('reference7')
                os.remove('reference8')
                os.remove('predictions')

In [ ]:
print('Seen domains:')
for _set in ['dev', 'test']:
    gold_path=os.path.join(path, 'data', 'end2end', _set + '.json')
    gold = json.load(open(gold_path))
    for kind in ['pipeline', 'end2end']:
        for model in ['rand', 'major', 'transformer', 'rnn']:
            if kind == 'end2end' and model in ['rand', 'major']:
                continue
            else:
                p = os.path.join(path, 'results', kind, model, _set + '.out.postprocessed')
                with open(p) as f:
                    y_pred_ = f.read().split('\n')[:-1]

                y_real, y_pred = [], []
                for i, g in enumerate(gold):
                    if g['category'] not in unseen_domains:
                        targets = [nltk.word_tokenize(' '.join(target['output'])) for target in g['targets']]
                        t = [' '.join(target).lower() for target in targets]
                        y_real.append(t)
                        pred = ' '.join(nltk.word_tokenize(y_pred_[i])).lower()
                        y_pred.append(pred)

                with open('predictions', 'w') as f:
                    f.write('\n'.join(y_pred))

                nfiles = max([len(refs) for refs in y_real])
                for i in range(nfiles):
                    with open('reference' + str(i+1), 'w') as f:
                        for refs in y_real:
                            if i < len(refs):
                                f.write(refs[i])
                            f.write('\n')

                nematus = '/roaming/tcastrof/workspace/nematus/data/multi-bleu.perl'
                command = 'perl ' + nematus + ' reference1 reference2 reference3 reference4 reference5 reference6 reference7 reference8 < predictions'
                result = subprocess.check_output(command, shell=True)
                print('Final: ')
                print('Set: ', _set)
                print('Approach:', kind)
                print('Model: ', model)
                print(result)
                print(10 * '-')

                os.remove('reference1')
                os.remove('reference2')
                os.remove('reference3')
                os.remove('reference4')
                os.remove('reference5')
                os.remove('reference6')
                os.remove('reference7')
                os.remove('reference8')
                os.remove('predictions')

In [ ]:
print('Unseen domains:')
for _set in ['test']:
    gold_path=os.path.join(path, 'data', 'end2end', _set + '.json')
    gold = json.load(open(gold_path))
    for kind in ['pipeline', 'end2end']:
        for model in ['rand', 'major', 'transformer', 'rnn']:
            if kind == 'end2end' and model in ['rand', 'major']:
                continue
            else:
                p = os.path.join(path, 'results', kind, model, _set + '.out.postprocessed')
                with open(p) as f:
                    y_pred_ = f.read().split('\n')[:-1]

                y_real, y_pred = [], []
                for i, g in enumerate(gold):
                    if g['category'] in unseen_domains:
                        targets = [nltk.word_tokenize(' '.join(target['output'])) for target in g['targets']]
                        t = [' '.join(target).lower() for target in targets]
                        y_real.append(t)
                        pred = ' '.join(nltk.word_tokenize(y_pred_[i])).lower()
                        y_pred.append(pred)

                with open('predictions', 'w') as f:
                    f.write('\n'.join(y_pred))

                nfiles = max([len(refs) for refs in y_real])
                for i in range(nfiles):
                    with open('reference' + str(i+1), 'w') as f:
                        for refs in y_real:
                            if i < len(refs):
                                f.write(refs[i])
                            f.write('\n')

                nematus = '/roaming/tcastrof/workspace/nematus/data/multi-bleu.perl'
                command = 'perl ' + nematus + ' reference1 reference2 reference3 reference4 reference5 reference6 reference7 reference8 < predictions'
                result = subprocess.check_output(command, shell=True)
                print('Final: ')
                print('Set: ', _set)
                print('Approach:', kind)
                print('Model: ', model)
                print(result)
                print(10 * '-')

                try:
                    os.remove('predictions')
                    os.remove('reference1')
                    os.remove('reference2')
                    os.remove('reference3')
                    os.remove('reference4')
                    os.remove('reference5')
                    os.remove('reference6')
                    os.remove('reference7')
                    os.remove('reference8')
                except:
                    pass

### Evaluation of Final Texts (METEOR)

In [ ]:
print('All domains:')
for _set in ['dev', 'test']:
    gold_path=os.path.join(path, 'data', 'end2end', _set + '.json')
    gold = json.load(open(gold_path))
    for kind in ['pipeline', 'end2end']:
        for model in ['rand', 'major', 'transformer', 'rnn']:
            if kind == 'end2end' and model in ['rand', 'major']:
                continue
            else:
                p = os.path.join(path, 'results', kind, model, _set + '.out.postprocessed')
                with open(p) as f:
                    y_pred_ = f.read().split('\n')[:-1]

                y_real, y_pred = [], []
                for i, g in enumerate(gold):
#                     if g['category'] in unseen_domains:
                    targets = [nltk.word_tokenize(' '.join(target['output'])) for target in g['targets']]
                    t = [' '.join(target).lower() for target in targets]
                    y_real.append(t)
                    pred = ' '.join(nltk.word_tokenize(y_pred_[i])).lower()
                    y_pred.append(pred)


                with open('predictions', 'w') as f:
                    f.write('\n'.join(y_pred))

                with open('reference', 'w') as f:
                    for refs in y_real:
                        for i in range(8):
                            if i < len(refs):
                                f.write(refs[i])
                            else:
                                f.write('')
                            f.write('\n')

                java = '/roaming/tcastrof/workspace/java/jre1.8.0_181/bin/java -Xmx2G -jar '
                java += '/home/tcastrof/workspace/meteor-1.5/meteor-1.5.jar predictions reference -l en -norm -r 8'
                result = subprocess.check_output(java, shell=True)
                print('Final: ')
                print('Set: ', _set)
                print('Approach:', kind)
                print('Model: ', model)
                print(result.split(b'\n')[-2])
                print(10 * '-')

                os.remove('reference')
                os.remove('predictions')


In [ ]:
print('Seen domains:')
for _set in ['dev', 'test']:
    gold_path=os.path.join(path, 'data', 'end2end', _set + '.json')
    gold = json.load(open(gold_path))
    for kind in ['pipeline', 'end2end']:
        for model in ['rand', 'major', 'transformer', 'rnn']:
            if kind == 'end2end' and model in ['rand', 'major']:
                continue
            else:
                p = os.path.join(path, 'results', kind, model, _set + '.out.postprocessed')
                with open(p) as f:
                    y_pred_ = f.read().split('\n')[:-1]

                y_real, y_pred = [], []
                for i, g in enumerate(gold):
                    if g['category'] not in unseen_domains:
                        targets = [nltk.word_tokenize(' '.join(target['output'])) for target in g['targets']]
                        t = [' '.join(target).lower() for target in targets]
                        y_real.append(t)
                        pred = ' '.join(nltk.word_tokenize(y_pred_[i])).lower()
                        y_pred.append(pred)


                with open('predictions', 'w') as f:
                    f.write('\n'.join(y_pred))

                with open('reference', 'w') as f:
                    for refs in y_real:
                        for i in range(8):
                            if i < len(refs):
                                f.write(refs[i])
                            else:
                                f.write('')
                            f.write('\n')

                java = '/roaming/tcastrof/workspace/java/jre1.8.0_181/bin/java -Xmx2G -jar '
                java += '/home/tcastrof/workspace/meteor-1.5/meteor-1.5.jar predictions reference -l en -norm -r 8'
                result = subprocess.check_output(java, shell=True)
                print('Final: ')
                print('Set: ', _set)
                print('Approach:', kind)
                print('Model: ', model)
                print(result.split(b'\n')[-2])
                print(10 * '-')

                os.remove('reference')
                os.remove('predictions')

In [ ]:
print('Unseen domains:')
for _set in ['test']:
    gold_path=os.path.join(path, 'data', 'end2end', _set + '.json')
    gold = json.load(open(gold_path))
    for kind in ['pipeline', 'end2end']:
        for model in ['rand', 'major', 'transformer', 'rnn']:
            if kind == 'end2end' and model in ['rand', 'major']:
                continue
            else:
                p = os.path.join(path, 'results', kind, model, _set + '.out.postprocessed')
                with open(p) as f:
                    y_pred_ = f.read().split('\n')[:-1]

                y_real, y_pred = [], []
                for i, g in enumerate(gold):
                    if g['category'] in unseen_domains:
                        targets = [nltk.word_tokenize(' '.join(target['output'])) for target in g['targets']]
                        t = [' '.join(target).lower() for target in targets]
                        y_real.append(t)
                        pred = ' '.join(nltk.word_tokenize(y_pred_[i])).lower()
                        y_pred.append(pred)


                with open('predictions', 'w') as f:
                    f.write('\n'.join(y_pred))

                with open('reference', 'w') as f:
                    for refs in y_real:
                        for i in range(8):
                            if i < len(refs):
                                f.write(refs[i])
                            else:
                                f.write('')
                            f.write('\n')

                java = '/roaming/tcastrof/workspace/java/jre1.8.0_181/bin/java -Xmx2G -jar '
                java += '/home/tcastrof/workspace/meteor-1.5/meteor-1.5.jar predictions reference -l en -norm -r 8'
                result = subprocess.check_output(java, shell=True)
                print('Final: ')
                print('Set: ', _set)
                print('Approach:', kind)
                print('Model: ', model)
                print(result.split(b'\n')[-2])
                print(10 * '-')

                os.remove('reference')
                os.remove('predictions')


### Evaluation of Final Texts (Fluency and Semantic)

In [ ]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h, m-h, m+h

path = 'evaluation/human/grades.json'
grades = json.load(open(path))

path = 'evaluation/human/participants.json'
participants = json.load(open(path))

In [ ]:
from collections import Counter
grades = json.load(open('evaluation/human/ngrades.json'))
participant_ids = set([w['participant_id'] for w in grades])
print('Number of participants: ', len(participant_ids))

participants = [p for p in participants if p['id'] in participant_ids]

print('***Gender:***')
print(Counter([p['gender'] for p in participants]))
print('***English Proficiency Level:***')
print(Counter([p['english_proficiency_level'] for p in participants]))
print('***Age:***')
print(round(np.mean([int(p['age']) for p in participants]), 2))
print('\n')

print('All Domains')
models = set([g['model'] for g in grades])
print('Fluency: ')
for model in ['rand', 'major', 'rnn', 'transformer', 'e2ernn', 'e2etransformer', 'melbourne', 'upfforge', 'original']:
    fluency = [float(g['fluency']) for g in grades if g['model'] == model]# and g['category'] in unseen_domains]
    print('{0}: {1} +-{2}'.format(model, round(np.mean(fluency), 2), round(mean_confidence_interval(fluency)[1], 2)))
print('\n')
print('Semantics: ')
for model in ['rand', 'major', 'rnn', 'transformer', 'e2ernn', 'e2etransformer', 'melbourne', 'upfforge', 'original']:
    semantic = [float(g['semantic']) for g in grades if g['model'] == model]# and g['category'] in unseen_domains]
    print('{0}: {1} +-{2}'.format(model, round(np.mean(semantic), 2), round(mean_confidence_interval(semantic)[1], 2)))

In [ ]:
print('Seen Domains')
models = set([g['model'] for g in grades])
print('Fluency: ')
for model in ['rand', 'major', 'rnn', 'transformer', 'e2ernn', 'e2etransformer', 'melbourne', 'upfforge', 'original']:
    fluency = [float(g['fluency']) for g in grades if g['model'] == model and g['category'] not in unseen_domains]
    print('{0}: {1} +-{2}'.format(model, round(np.mean(fluency), 2), round(mean_confidence_interval(fluency)[1], 2)))
print('\n')
print('Semantics: ')
for model in ['rand', 'major', 'rnn', 'transformer', 'e2ernn', 'e2etransformer', 'melbourne', 'upfforge', 'original']:
    semantic = [float(g['semantic']) for g in grades if g['model'] == model and g['category'] not in unseen_domains]
    print('{0}: {1} +-{2}'.format(model, round(np.mean(semantic), 2), round(mean_confidence_interval(semantic)[1], 2)))

In [ ]:
print('Unseen Domains')
models = set([g['model'] for g in grades])
print('Fluency: ')
for model in ['rand', 'major', 'rnn', 'transformer', 'e2ernn', 'e2etransformer', 'melbourne', 'upfforge', 'original']:
    fluency = [float(g['fluency']) for g in grades if g['model'] == model and g['category'] in unseen_domains]
    print('{0}: {1} +-{2}'.format(model, round(np.mean(fluency), 2), round(mean_confidence_interval(fluency)[1], 2)))
print('\n')
print('Semantics: ')
for model in ['rand', 'major', 'rnn', 'transformer', 'e2ernn', 'e2etransformer', 'melbourne', 'upfforge', 'original']:
    semantic = [float(g['semantic']) for g in grades if g['model'] == model and g['category'] in unseen_domains]
    print('{0}: {1} +-{2}'.format(model, round(np.mean(semantic), 2), round(mean_confidence_interval(semantic)[1], 2)))

In [ ]:
from scipy.stats import mannwhitneyu, wilcoxon

models = ['rand', 'major', 'rnn', 'transformer', 'e2ernn', 'e2etransformer', 'melbourne', 'upfforge', 'original']
for i, model1 in enumerate(models):
    for j, model2 in enumerate(models):
        if model1 != model2:
            fluency1 = [float(g['semantic']) for g in grades if g['model'] == model1]# and g['category'] in unseen_domains]
            fluency2 = [float(g['semantic']) for g in grades if g['model'] == model2]# and g['category'] in unseen_domains]
            print(model1, 'x', model2, ':', round(mannwhitneyu(fluency1, fluency2)[1], 2) < 0.05)

### Evaluation of Annotations

In [ ]:
gold_path='evaluation/questionaire/trials/gold.json'
gold = json.load(open(gold_path))

path='evaluation/questionaire/annotations.json'
annotations = json.load(open(path))


In [ ]:
for model in annotations:
    values = {
        'detmistake': 0,
        'fluency': [],
        'moreinformation': 0,
        'numpreds': 0,
        'referencemistake': 0,
        'semantics': [],
        'structurefollowed': 0,
        'verbmistake': 0
    }
    dem = 0
    for trial in annotations[model]:
        g = [g for g in gold if g['eid'] == trial][0]
        category = g['category']
        size = g['size']
#         if category in unseen_domains:
        if annotations[model][trial]['structurefollowed']:
            values['structurefollowed'] += 1
        if annotations[model][trial]['moreinformation']:
            values['moreinformation'] += 1
        if annotations[model][trial]['referencemistake']:
            values['referencemistake'] += 1
        if annotations[model][trial]['verbmistake']:
            values['verbmistake'] += 1
        if annotations[model][trial]['detmistake']:
            values['detmistake'] += 1
        if int(size) == int(annotations[model][trial]['numpreds']):
            values['numpreds'] += 1
        dem += 1
        values['fluency'].append(float(annotations[model][trial]['fluency']))
        values['semantics'].append(float(annotations[model][trial]['semantics']))
    values['structurefollowed'] /= dem
    values['moreinformation'] /= dem
    values['referencemistake'] /= dem
    values['verbmistake'] /= dem
    values['detmistake'] /= dem

    print('Model: ', model)
    print('Structured followed in {0} of the cases'.format(round(values['structurefollowed'], 2)))
    print('More information in {0} of the cases'.format(round(values['moreinformation'], 2)))
    print('Exact number of predicates in {0} out of {1} of the cases ({2})'.format(values['numpreds'], dem, round(values['numpreds']/dem, 2)))
    print('Reference mistakes in {0} of the cases'.format(round(values['referencemistake'], 2)))
    print('Verb mistakes in {0} of the cases'.format(round(values['verbmistake'], 2)))
    print('Determiner mistakes in {0} of the cases'.format(round(values['detmistake'], 2)))
    print('Fluency: {0}'.format(np.mean(values['fluency'])))
    print('Semantics: {0}'.format(np.mean(values['semantics'])))
    print(10 * '-')

#### Inter-annotator Agreement


In [ ]:
import os
from nltk.metrics.agreement import AnnotationTask

def kappa(obs):
    t = AnnotationTask(obs)
    print("\nWeighted kappa as per NLTK:\t", t.weighted_kappa(),
          "\nRegular kappa as per NLTK:\t", t.kappa(),
          "\nKrippendorff alpha as per NLTK:\t", t.alpha(),
          "\n===========================================\n")

path='evaluation/questionaire/observations/'
if not os.path.exists(path):
    os.mkdir(path)

ann1_model1 = annotations['models\\model1.xml']
ann2_model1 = annotations['model1.xml']
ann1_model3 = annotations['models\\model3.xml']
ann2_model3 = annotations['model3.xml']

obs_fluency = []
for trial in ann1_model1:
    obs_fluency.append(('ann1', 'model1_'+trial, int(ann1_model1[trial]['fluency'])))
for trial in ann1_model3:
    obs_fluency.append(('ann1', 'model3_'+trial, int(ann1_model3[trial]['fluency'])))
for trial in ann2_model1:
    obs_fluency.append(('ann2', 'model1_'+trial, int(ann2_model1[trial]['fluency'])))
for trial in ann2_model3:
    obs_fluency.append(('ann2', 'model3_'+trial, int(ann2_model3[trial]['fluency'])))

print('Fluency:')
kappa(obs_fluency)

with open(os.path.join(path, 'fluency.csv'), 'w') as f:
    for i, o in enumerate(obs_fluency):
        obs_fluency[i] = list(obs_fluency[i])
        obs_fluency[i][2] = str(obs_fluency[i][2])
        obs_fluency[i] = ','.join(obs_fluency[i])
    f.write('\n'.join(obs_fluency))
########################################################################################################
obs_semantic = []
for trial in ann1_model1:
    obs_semantic.append(('ann1', 'model1_'+trial, int(ann1_model1[trial]['semantics'])))
for trial in ann1_model3:
    obs_semantic.append(('ann1', 'model3_'+trial, int(ann1_model3[trial]['semantics'])))
for trial in ann2_model1:
    obs_semantic.append(('ann2', 'model1_'+trial, int(ann2_model1[trial]['semantics'])))
for trial in ann2_model3:
    obs_semantic.append(('ann2', 'model3_'+trial, int(ann2_model3[trial]['semantics'])))

print('Semantic:')
kappa(obs_semantic)

with open(os.path.join(path, 'semantic.csv'), 'w') as f:
    for i, o in enumerate(obs_semantic):
        obs_semantic[i] = list(obs_semantic[i])
        obs_semantic[i][2] = str(obs_semantic[i][2])
        obs_semantic[i] = ','.join(obs_semantic[i])
    f.write('\n'.join(obs_semantic))
########################################################################################################
obs_preds = []
for trial in ann1_model1:
    obs_preds.append(('1', 'model1_'+trial, int(ann1_model1[trial]['numpreds'])))
for trial in ann1_model3:
    obs_preds.append(('1', 'model3_'+trial, int(ann1_model3[trial]['numpreds'])))
for trial in ann2_model1:
    obs_preds.append(('2', 'model1_'+trial, int(ann2_model1[trial]['numpreds'])))
for trial in ann2_model3:
    obs_preds.append(('2', 'model3_'+trial, int(ann2_model3[trial]['numpreds'])))

print('Predicates:')
kappa(obs_preds)

with open(os.path.join(path, 'predicates.csv'), 'w') as f:
    for i, o in enumerate(obs_preds):
        obs_preds[i] = list(obs_preds[i])
        obs_preds[i][2] = str(obs_preds[i][2])
        obs_preds[i] = ','.join(obs_preds[i])
    f.write('\n'.join(obs_preds))
########################################################################################################
obs = []
for trial in ann1_model1:
    obs.append(('1', 'model1_'+trial, 1 if ann1_model1[trial]['structurefollowed'] else 0))
for trial in ann1_model3:
    obs.append(('1', 'model3_'+trial, 1 if ann1_model3[trial]['structurefollowed'] else 0))
for trial in ann2_model1:
    obs.append(('2', 'model1_'+trial, 1 if ann2_model1[trial]['structurefollowed'] else 0))
for trial in ann2_model3:
    obs.append(('2', 'model3_'+trial, 1 if ann2_model3[trial]['structurefollowed'] else 0))

print('Structure Followed:')
kappa(obs)

with open(os.path.join(path, 'structfollowed.csv'), 'w') as f:
    for i, o in enumerate(obs):
        obs[i] = list(obs[i])
        obs[i][2] = str(obs[i][2])
        obs[i] = ','.join(obs[i])
    f.write('\n'.join(obs))
########################################################################################################
obs = []
for trial in ann1_model1:
    obs.append(('1', 'model1_'+trial, 1 if ann1_model1[trial]['moreinformation'] else 0))
for trial in ann1_model3:
    obs.append(('1', 'model3_'+trial, 1 if ann1_model3[trial]['moreinformation'] else 0))
for trial in ann2_model1:
    obs.append(('2', 'model1_'+trial, 1 if ann2_model1[trial]['moreinformation'] else 0))
for trial in ann2_model3:
    obs.append(('2', 'model3_'+trial, 1 if ann2_model3[trial]['moreinformation'] else 0))

print('Overgeneration:')
kappa(obs)

with open(os.path.join(path, 'overgeneration.csv'), 'w') as f:
    for i, o in enumerate(obs):
        obs[i] = list(obs[i])
        obs[i][2] = str(obs[i][2])
        obs[i] = ','.join(obs[i])
    f.write('\n'.join(obs))
########################################################################################################
obs = []
for trial in ann1_model1:
    obs.append(('1', 'model1_'+trial, 1 if ann1_model1[trial]['verbmistake'] else 0))
for trial in ann1_model3:
    obs.append(('1', 'model3_'+trial, 1 if ann1_model3[trial]['verbmistake'] else 0))
for trial in ann2_model1:
    obs.append(('2', 'model1_'+trial, 1 if ann2_model1[trial]['verbmistake'] else 0))
for trial in ann2_model3:
    obs.append(('2', 'model3_'+trial, 1 if ann2_model3[trial]['verbmistake'] else 0))

print('Verb mistakes:')
kappa(obs)

with open(os.path.join(path, 'verbmistakes.csv'), 'w') as f:
    for i, o in enumerate(obs):
        obs[i] = list(obs[i])
        obs[i][2] = str(obs[i][2])
        obs[i] = ','.join(obs[i])
    f.write('\n'.join(obs))
########################################################################################################
obs = []
for trial in ann1_model1:
    obs.append(('1', 'model1_'+trial, 1 if ann1_model1[trial]['detmistake'] else 0))
for trial in ann1_model3:
    obs.append(('1', 'model3_'+trial, 1 if ann1_model3[trial]['detmistake'] else 0))
for trial in ann2_model1:
    obs.append(('2', 'model1_'+trial, 1 if ann2_model1[trial]['detmistake'] else 0))
for trial in ann2_model3:
    obs.append(('2', 'model3_'+trial, 1 if ann2_model3[trial]['detmistake'] else 0))

print('Determiner mistakes:')
kappa(obs)

with open(os.path.join(path, 'detmistakes.csv'), 'w') as f:
    for i, o in enumerate(obs):
        obs[i] = list(obs[i])
        obs[i][2] = str(obs[i][2])
        obs[i] = ','.join(obs[i])
    f.write('\n'.join(obs))
########################################################################################################
obs = []
for trial in ann1_model1:
    obs.append(('1', 'model1_'+trial, 1 if ann1_model1[trial]['referencemistake'] else 0))
for trial in ann1_model3:
    obs.append(('1', 'model3_'+trial, 1 if ann1_model3[trial]['referencemistake'] else 0))
for trial in ann2_model1:
    obs.append(('2', 'model1_'+trial, 1 if ann2_model1[trial]['referencemistake'] else 0))
for trial in ann2_model3:
    obs.append(('2', 'model3_'+trial, 1 if ann2_model3[trial]['referencemistake'] else 0))

print('Reference mistakes:')
kappa(obs)

with open(os.path.join(path, 'refmistakes.csv'), 'w') as f:
    for i, o in enumerate(obs):
        obs[i] = list(obs[i])
        obs[i][2] = str(obs[i][2])
        obs[i] = ','.join(obs[i])
    f.write('\n'.join(obs))

In [ ]:
def split_triples(text):
    triples, triple = [], []
    for w in text:
        if w not in ['<TRIPLE>', '</TRIPLE>']:
            triple.append(w)
        elif w == '</TRIPLE>':
            triples.append(triple)
            triple = []
    return triples

def ordering_analysis(ordering, gold):
    for i, entry in enumerate(gold):
        triples = split_triples(entry['source'])

        num, visited = 0, []
        for triple in triples:
            for j, predicate in enumerate(ordering[i]):
                if predicate == triple[1] and j not in visited:
                    num += 1
                    visited.append(j)
        # How many predicates in the modified tripleset are present in the result?
        entry['ordering'] = num
    return gold


def structing_analysis(structing, gold):
    for i, entry in enumerate(self.gold):
        triples = split_triples(entry['source'])

        num, visited = 0, []
        for triple in triples:
            for j, predicate in enumerate(structing[i]):
                if predicate == triple[1] and j not in visited:
                    num += 1
                    visited.append(j)
        # How many predicates in the modified tripleset are present in the result?
        entry['structing'] = num
    return gold

In [ ]:
path = 'results/questionaire/partial.txt'
with open(path) as f:
    eids = f.read().split()

gold_path='evaluation/questionaire/gold.json'
gold = json.load(open(gold_path))

p = 'evaluation/results/pipeline/transformer/test.structing.postprocessed'
with open(p) as f:
    ordering = f.read().split('\n')[:-1]

pos, dem = 0, 0
for i, entry in enumerate(gold):
    if entry['eid'] in eids:
        category = entry['category']
#         if category in unseen_domains:
        triples = split_triples(entry['source'])
        num, visited = 0, []
        for triple in triples:
            for j, predicate in enumerate(ordering[i].split()):
                if predicate == triple[1] and j not in visited:
                    num += 1
                    visited.append(j)
                if len(visited) == len(triples):
                    break
        if len(triples) == len(visited):
            pos += 1
        dem += 1

print('Exact number of predicates in {0} out of {1} of the cases ({2})'.format(pos, dem, round(pos/dem, 2)))
